In [ ]:
!git clone https://github.com/RIPS-2024-Aerospace/Aerospace-Project.git

fatal: destination path 'Aerospace-Project' already exists and is not an empty directory.


In [ ]:
!pip install simanneal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from simanneal import Annealer

%run "./Aerospace-Project/Standard Filters/DiffKf.ipynb"
%run "./Aerospace-Project/Standard Filters/KF.ipynb"

In [35]:
# Bhattacharyya distance function
def bhattacharyya_distance(mu1, mu2, Sigma1, Sigma2):
    Sigma = (Sigma1 + Sigma2) / 2
    inv_Sigma = np.linalg.inv(Sigma)
    term1 = 1/8 * np.dot(np.dot((mu1 - mu2).T, inv_Sigma), (mu1 - mu2))
    term2 = 1/2 * np.log(np.linalg.det(Sigma) / np.sqrt(np.linalg.det(Sigma1) * np.linalg.det(Sigma2)))
    return term1 + term2

# Function to run filters and return covariances
def run_filters(W):
    print(W)
    dt = 10

    # define C
    C_adj = np.array([[1, 1, 0, 0, 1],
                      [1, 1, 1, 0, 0],
                      [0, 1, 1, 1, 0],
                      [0, 0, 1, 1, 1],
                      [1, 0, 0, 1, 1]])
    C = C_adj * np.reshape(W, (5, 5))
    C_unweighted = np.array([[1 if x != 0 else 0 for x in row] for row in C])
    num_stns = len(C[0])

    A = np.array([[1, dt, 0, 0], [0, 1, 0, 0],[0,0,1,dt], [0, 0, 0, 1]])
    H = np.array([[1, 0, 0, 0],[0,0,1,0]])

    dkf_state_size = len(A)
    dkf_measure_size = len(H)

    q = 0.001
    Q = q*np.array([[(dt**3)/3, (dt**2)/2, 0, 0], [(dt**2)/2, dt, 0, 0],[0,0,(dt**3)/3,(dt**2)/2], [0, 0, (dt**2)/2, dt]])
    R = np.array([[4,0],[0,4]])

    A_kf = np.kron(np.eye(num_stns), A)
    H_kf = np.kron(np.eye(num_stns), H)
    Q_kf = np.kron(np.eye(num_stns), Q)
    R_kf = np.kron(np.eye(num_stns), R)

    kf_state_size = A_kf.shape[0]
    kf_measure_size = R_kf.shape[0]

    F = [A for _ in range(num_stns)]
    G = [np.eye(A.shape[0]) for _ in range(num_stns)]
    H_dkf = [H for _ in range(num_stns)]

    Q_dkf = [Q for _ in range(num_stns)]
    R_dkf = [R for _ in range(num_stns)]

    procc_noise_kf = lambda : np.linalg.cholesky(Q_kf) @ np.random.normal(np.array([[0 for _ in range(kf_state_size)]]).T)
    measure_noise_kf = lambda : np.linalg.cholesky(R_kf) @ np.random.normal(np.array([[0 for _ in range(kf_measure_size)]]).T)

    measure_kf_to_dkf  = lambda z: [np.array([z[H.shape[0]*i + j] for j in range(H.shape[0])]) for i in range(num_stns)]
    state_kf_to_dkf = lambda z: [np.array([z[A.shape[0]*i + j] for j in range(A.shape[0])]) for i in range(num_stns)]

    # True Initial
    x0_kf = np.array([[np.random.normal(0, np.sqrt(Q_kf[i, i])) for i in range(kf_state_size)]]).T

    # Initial Estimate
    x_kf = np.array([[np.random.normal(0, 5) for i in range(kf_state_size)]]).T
    x_dkf = state_kf_to_dkf(x_kf)

    P_kf = 10 * np.copy(Q_kf)
    P_dkf = [10 * np.copy(Q) for _ in range(num_stns)]

    kf = KalmanFilter(A=A_kf, H=H_kf, Q=Q_kf, R=R_kf, P=P_kf, x0=x0_kf)
    dkf = DiffKF(C, F, G, H_dkf, R_dkf, Q_dkf, x_dkf, P_dkf)

    iters = 60

    truth = np.zeros((iters + 1, kf_state_size, 1))
    truth[0] = x0_kf

    measurements = np.zeros((iters + 1, kf_measure_size, 1))
    measurements[0] = (H_kf @ x0_kf) + measure_noise_kf()

    predictions_kf = np.zeros((iters, kf_state_size, 1))
    predictions_dkf = np.zeros((iters, num_stns, A.shape[0], 1))

    errors_kf = np.zeros((iters, kf_state_size, 1))
    errors_dkf = np.zeros((iters, num_stns, A.shape[0], 1))

    P_hist_kf = np.zeros((iters, kf_state_size, kf_state_size))
    P_hist_dkf = np.zeros((iters, num_stns, A.shape[0], A.shape[0]))
    full_system_P_hist = np.zeros((iters, kf_state_size, kf_state_size))
    prev_cov = np.block([[np.zeros(P_dkf[0].shape) if i != j else dkf.nodes[i].P for j in range(num_stns)] for i in range(num_stns)])

    for i in range(iters):
        kf.update(measurements[i])
        dkf.update(measure_kf_to_dkf(measurements[i]))

        predictions_dkf[i] = [dkf.nodes[j].x for j in range(num_stns)]
        errors_dkf[i] = [dkf.nodes[j].x - state_kf_to_dkf(truth[i])[j] for j in range(num_stns)]
        station_covs = [dkf.nodes[j].P for j in range(num_stns)]
        P_hist_dkf[i] = station_covs

        prev_cov = get_diff_cov(prev_cov, station_covs, dkf, num_stns, A, H, Q, R, C, C_unweighted, G)
        full_system_P_hist[i] = prev_cov

        predictions_kf[i] = kf.x
        errors_kf[i] = kf.x - truth[i]
        P_hist_kf[i] = kf.P

        kf.predict()
        dkf.predict()

        truth[i + 1] = A_kf @ x0_kf + procc_noise_kf()
        measurements[i + 1] = H_kf @ truth[i + 1] + measure_noise_kf()

    return (P_hist_kf[40], full_system_P_hist[40])

# Function to get diffusion covariance
def get_diff_cov(prev_cov, Station_cov, dkf, num_stns, A, H, Q, R, C, C_unweighted, G):
    S = lambda i: np.sum([node.H.T @ np.linalg.inv(node.R) @ node.H for node in dkf.nodes[i].nbhrs], axis=0)

    S_full = np.block([[np.zeros(A.shape) if i != j else S(j) for j in range(num_stns)] for i in range(num_stns)])
    H_full = np.kron(np.eye(num_stns), H)
    P_full = np.block([[np.zeros(Station_cov[0].shape) if i != j else Station_cov[j] for j in range(num_stns)] for i in range(num_stns)])
    R_full = np.kron(np.eye(num_stns), R)

    C_full = np.kron(C, np.eye(A.shape[0]))
    A_full = np.kron(C_unweighted, np.eye(A.shape[0]))

    F_i = C_full.T @ (np.eye(S_full.shape[1]) - (P_full @ S_full)) @ np.kron(np.eye(num_stns), A)
    G_i = C_full.T @ (np.eye(S_full.shape[1]) - (P_full @ S_full)) @ np.kron(np.eye(num_stns), G[0])
    D_i = C_full.T @ P_full @ A_full.T @ H_full.T @ np.linalg.inv(R_full)

    term1 = (F_i @ prev_cov @ F_i.T)
    term2 = G_i @ np.kron(np.ones((num_stns, num_stns)), Q) @ G_i.T
    term3 = D_i @ R_full @ D_i.T

    return term1 + term2 + term3

In [43]:
# Define the cost function for optimization
def cost_func(diffusion_weights):
    dt = 10
    num_stns = 5

    A = np.array([[1, dt, 0, 0], [0, 1, 0, 0], [0, 0, 1, dt], [0, 0, 0, 1]])
    H = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])
    q = 0.001
    Q = q * np.array([[(dt**3) / 3, (dt**2) / 2, 0, 0],
                      [(dt**2) / 2, dt, 0, 0],
                      [0, 0, (dt**3) / 3, (dt**2) / 2],
                      [0, 0, (dt**2) / 2, dt]])
    R = np.array([[4, 0], [0, 4]])

    # size of state and measurement vectors
    kf_state_size = A.shape[0] * num_stns
    kf_measure_size = H.shape[0] * num_stns

    # initialize the means to 0
    mu1 = np.zeros(kf_state_size)
    mu2 = np.zeros(kf_state_size)

    # run filters to get the sigma tuple
    sigmas = run_filters(diffusion_weights)
    Sigma1 = sigmas[0] # state covariance of centralized KF
    Sigma2 = sigmas[1] # state covariance of diffusion KF
    return bhattacharyya_distance(mu1, mu2, Sigma1, Sigma2)

# Simulated Annealing class for optimization
class AnnealOptimization(Annealer):
    def __init__(self, state, C_adj):
        super(AnnealOptimization, self).__init__(state)
        self.steps = 100
        self.C_adj = C_adj

    def move(self):
        # Randomly modify one of the weights corresponding to a non-zero element in self.C_adj
        idx = np.random.choice(np.where(self.C_adj.flatten() == 1)[0])
        self.state[idx] += np.random.normal(0, 0.1)

        # Constraint: weights must be non-negative
        self.state[self.state < 0] = 0

        # Constraint: row stochasticity for non-zero elements
        for i in range(0, len(self.state), 5):
            row = self.state[i:i + 5] * self.C_adj[i//5]
            row_sum = np.sum(row)
            if row_sum != 0:
                self.state[i:i + 5] = (row / row_sum) * self.C_adj[i//5]

    def energy(self):
        return cost_func(self.state)

# Run the optimization
def run_optimize():
    C = np.array([[0.34, 0.33, 0, 0, 0.33],
                  [0.33, 0.34, 0.33, 0, 0],
                  [0, 0.33, 0.34, 0.33, 0],
                  [0, 0, 0.33, 0.34, 0.33],
                  [0.33, 0, 0, 0.33, 0.34]])

    C_adj = np.array([[1, 1, 0, 0, 1],
                      [1, 1, 1, 0, 0],
                      [0, 1, 1, 1, 0],
                      [0, 0, 1, 1, 1],
                      [1, 0, 0, 1, 1]])

    # Flatten C matrix to use as initial weights
    weights = C.flatten()

    # Initialize simulated annealing optimization
    annealer = AnnealOptimization(weights, C_adj)
    best_state, best_energy = annealer.anneal()

    # Checks if each row in the final best weights matrix sums to 1
    best_weights_matrix = np.reshape(best_state, (5, 5))
    row_sums = np.sum(best_weights_matrix, axis=1)
    print(f'Best weights matrix:\n{best_weights_matrix}')
    print(f'Row sums: {row_sums}')
    print(f'Best Bhattacharyya distance: {best_energy}')

run_optimize()

[0.34 0.33 0.   0.   0.33 0.33 0.34 0.33 0.   0.   0.   0.33 0.34 0.33
 0.   0.   0.   0.33 0.34 0.33 0.33 0.   0.   0.33 0.34]


 Temperature        Energy    Accept   Improve     Elapsed   Remaining


[0.34       0.33       0.         0.         0.33       0.33
 0.34       0.33       0.         0.         0.         0.33
 0.34       0.33       0.         0.         0.         0.38071051
 0.31426631 0.30502318 0.33       0.         0.         0.33
 0.34      ]


[0.34       0.33       0.         0.         0.33       0.33
 0.34       0.33       0.         0.         0.         0.32682064
 0.34635873 0.32682064 0.         0.         0.         0.38071051
 0.31426631 0.30502318 0.33       0.         0.         0.33
 0.34      ]


[0.34       0.33       0.         0.         0.33       0.33
 0.34       0.33       0.         0.         0.         0.32682064
 0.34635873 0.32682064 0.         0.         0.         0.32093047
 0.34460245 0.33446708 0.33       0.         0.         0.33
 0.34      ]


...

[0.55675824 0.29860597 0.         0.         0.14463579 0.16473023
 0.36048539 0.47478438 0.         0.         0.         0.2338003
 0.08982473 0.67637497 0.         0.         0.         0.00368084
 0.66562896 0.3306902  0.19574458 0.         0.         0.49606905
 0.30818637]


[0.55675824 0.29860597 0.         0.         0.14463579 0.16473023
 0.36048539 0.47478438 0.         0.         0.         0.2338003
 0.08982473 0.67637497 0.         0.         0.         0.00368084
 0.66562896 0.3306902  0.19769958 0.         0.         0.49103604
 0.31126439]


[0.55675824 0.29860597 0.         0.         0.14463579 0.16473023
 0.36048539 0.47478438 0.         0.         0.         0.21704388
 0.08338702 0.6995691  0.         0.         0.         0.00368084
 0.66562896 0.3306902  0.19769958 0.         0.         0.49103604
 0.31126439]


[0.55675824 0.29860597 0.         0.         0.14463579 0.17935716
 0.35417271 0.46647013 0.         0.         0.         0.21704388
 0.08338702 0.6995691  0.         0.         0.         0.00368084
 0.66562896 0.3306902  0.19769958 0.         0.         0.49103604
 0.31126439]


Best weights matrix:
[[0.06633926 0.46683037 0.         0.         0.46683037]
 [0.41398244 0.29738205 0.28863551 0.         0.        ]
 [0.         0.42063251 0.15873499 0.42063251 0.        ]
 [0.         0.         0.26651615 0.37186657 0.36161728]
 [0.27556077 0.         0.         0.35681335 0.36762588]]
Row sums: [1. 1. 1. 1. 1.]
Best Bhattacharyya distance: 17.720442867315764
